# 라이브러리

In [148]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [149]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [150]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [151]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [152]:
table(data$target)


경영·행정·사무직             기타         서비스직 
            8961             7474             8544 

In [153]:
data <- subset(data, data$target == "기타")
table(data$target)


기타 
7474 

In [154]:
data$target<- ifelse(data$취업직종대분류=="제조 단순직","제조 단순직",data$취업직종대분류)
data$target <- ifelse(data$취업직종대분류 %in% c("보건·의료직","돌봄 서비스직(간병·육아)","사회복지·종교직"), "간호·보건 및 돌봄 서비스 관련 직군", data$target)
data$target <- ifelse(data$target %in% c("간호·보건 및 돌봄 서비스 관련 직군","제조 단순직"),  data$target,"전문·생산 및 정비 관련 직군")
table(data$target)


간호·보건 및 돌봄 서비스 관련 직군        전문·생산 및 정비 관련 직군 
                              2291                               2709 
                       제조 단순직 
                              2474 

In [155]:
data <- subset(data, data$target == "간호·보건 및 돌봄 서비스 관련 직군")
table(data$target)


간호·보건 및 돌봄 서비스 관련 직군 
                              2291 

In [156]:
data$target<- ifelse(data$취업직종대분류=="보건·의료직","보건·의료직",data$취업직종대분류)
data$target <- ifelse(data$취업직종대분류 %in% c("돌봄 서비스직(간병·육아)"), "돌봄 서비스직(간병·육아)", data$target)
data$target <- ifelse(data$target %in% c("돌봄 서비스직(간병·육아)","보건·의료직"),data$target,"사회복지·종교직")
table(data$target)


돌봄 서비스직(간병·육아)              보건·의료직          사회복지·종교직 
                     452                     1609                      230 

In [157]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [158]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [159]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [160]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [161]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [162]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	2291 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 10 10 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 3 4 5 3 5 5 1 2 3 3 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 8 2 7 2 2 2 5 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 6 10 1 1 1 1 1 1 6 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 1 1 1 1 1 1 1 1 1 ...
 $ target           : chr  "보건·의료직" "보건·의료직" "보건·의료직" "보건·의료직" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40           : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50           : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증    : logi  FALSE FALSE FALSE FALSE

[1] 0

In [163]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	2291 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 10 10 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 3 4 5 3 5 5 1 2 3 3 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 8 2 7 2 2 2 5 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 6 10 1 1 1 1 1 1 6 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 1 1 1 1 1 1 1 1 1 ...
 $ target           : chr  "보건·의료직" "보건·의료직" "보건·의료직" "보건·의료직" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40           : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50           : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증    : logi  FALSE FALSE FALSE FALSE

[1] 0

In [164]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [165]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	2291 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 10 10 9 ...
 $ target        : chr  "보건·의료직" "보건·의료직" "보건·의료직" "보건·의료직" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40        : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50        : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FALSE FALSE F

[1] 0

In [166]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [167]:
str(data_filtered_subset)

'data.frame':	2291 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 9 9 9 9 9 9 9 10 10 9 ...
 $ target        : chr  "보건·의료직" "보건·의료직" "보건·의료직" "보건·의료직" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40        : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50        : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : logi  FALSE FALSE F

In [168]:
summary(data_filtered_subset)

                  취업직종대분류    target            age_20       
 보건·의료직             :1609   Length:2291        Mode :logical  
 돌봄 서비스직(간병·육아): 452   Class :character   FALSE:1769     
 사회복지·종교직         : 230   Mode  :character   TRUE :522      
 건설·채굴직             :   0                                     
 경영·행정·사무직        :   0                                     
 경호·경비직             :   0                                     
 (Other)                 :   0                                     
   age_30          age_40          age_50          age_60       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2020      FALSE:1934      FALSE:1761      FALSE:1813     
 TRUE :271       TRUE :357       TRUE :530       TRUE :478      
                                                                
                                                                
                                                                
                                               

# 변동사항

In [169]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [170]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [171]:
str(data_filtered_subset)

'data.frame':	2291 obs. of  46 variables:
 $ target       : Factor w/ 3 levels "돌봄 서비스직(간병·육아)",..: 2 2 2 2 2 2 2 3 3 2 ...
 $ age_20       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_40       : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ age_50       : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE TRUE FALSE TRUE TRUE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  TRUE FALSE TRUE FALSE TRUE TRUE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FALS

In [172]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
         0.786530           0.071675           0.018083           0.048655  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
        -0.021835          -0.118497                 NA           0.319780  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
         0.192699           0.429710           0.208468           0.402022  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
        -0.112791          -0.165623           0.311216                 NA  
정신장애_중증TRUE           경북TRUE           강원TRUE           전남TRUE  
               NA           0.906982           0.933689           1.341351  
         충북TRUE           경기TRUE           인천TRUE           서울TRUE  
         0.747975           0.941989           0.974709           0.984180  
         울산TRUE           대전TRUE           부산TRUE           전북TRUE  
     

In [173]:
summary(data_filtered_subset)

                      target       age_20          age_30       
 돌봄 서비스직(간병·육아): 452   Mode :logical   Mode :logical  
 보건·의료직             :1609   FALSE:1769      FALSE:2020     
 사회복지·종교직         : 230   TRUE :522       TRUE :271      
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:1934      FALSE:1761      FALSE:1813      FALSE:2158     
 TRUE :357       TRUE :530       TRUE :478       TRUE :133      
 시각장애_중증   시각장애_경증   지체장애_중증   지체장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:1222      FALSE:2159      FALSE:2208      FALSE:1960     
 TRUE :1069      TRUE :132       TRUE :83        TRUE :331      
 지적장애_경증   지적장애_중증   청각장애_경증   청각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2289      FALSE:1754      FALSE:2244      FALSE:2268     
 TRUE :2         TRUE :537       TRUE :47        TRUE :23       
 정신장애_경증   정신장애_중증      경북    

# 시험준비

In [174]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [175]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [176]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [177]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=50)

In [178]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [179]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [180]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =10)

# weights:  493


initial  value 2081.891450 
iter  10 value 724.085545
iter  20 value 527.799125
iter  30 value 423.018139
iter  40 value 355.589912
iter  50 value 316.717970
iter  60 value 297.023037
iter  70 value 283.012653
iter  80 value 274.162551
iter  90 value 268.577725
iter 100 value 262.862460
final  value 262.862460 
stopped after 100 iterations


# 검증하기

In [181]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [182]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [183]:
# KNN 검증하기
# mean(knnmodel==y) 

In [184]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [185]:
#  RF 예측력
mean(predrf == y)

[1] 0.8163265

In [186]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.7842566

In [187]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.8134111

In [188]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.7900875

# 오분류표

In [189]:
#  Random Forest 오분류표
table(predrf,y)

                          y
predrf                     돌봄 서비스직(간병·육아) 보건·의료직 사회복지·종교직
  돌봄 서비스직(간병·육아)                       90          27              11
  보건·의료직                                    38         446              34
  사회복지·종교직                                 7           9              24

In [190]:
#  SVM 오분류표
table(predmodelsvm,y)

                          y
predmodelsvm               돌봄 서비스직(간병·육아) 보건·의료직 사회복지·종교직
  돌봄 서비스직(간병·육아)                       68          12               6
  보건·의료직                                    67         470              63
  사회복지·종교직                                 0           0               0

In [191]:
#  KNN 오분류표
# table(y, knnmodel)

In [192]:
# nnet 오분류표
table(prednnet,y)

                          y
prednnet                   돌봄 서비스직(간병·육아) 보건·의료직 사회복지·종교직
  돌봄 서비스직(간병·육아)                       90          33              19
  보건·의료직                                    34         425              23
  사회복지·종교직                                11          24              27

In [193]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,돌봄 서비스직(간병·육아),보건·의료직,사회복지·종교직
1,0,1,0
